## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [2]:
# 設定 data_path
dir_data = './data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [3]:
app_train['DAYS_EMPLOYED'] = abs(app_train['DAYS_EMPLOYED'])
sub_data = app_train[["AMT_INCOME_TOTAL", "DAYS_EMPLOYED", "DAYS_BIRTH"]] # subset
# day to year
sub_data["Years_EMPLOYED"] = sub_data['DAYS_EMPLOYED'] / 365
sub_data['Years_BIRTH'] = sub_data['DAYS_BIRTH'] / 365
sub_data.head()

,AMT_INCOME_TOTAL,DAYS_EMPLOYED,DAYS_BIRTH,Years_EMPLOYED,Years_BIRTH
0,202500.0,637.0,9461,1.745205,25.920548
1,270000.0,1188.0,16765,3.254795,45.931507
2,67500.0,225.0,19046,0.616438,52.180822
3,135000.0,3039.0,19005,8.326027,52.068493
4,121500.0,3038.0,19932,8.323288,54.608219


#### 等寬劃分 & 等頻劃分

In [4]:
sub_data["equal_width_Years_EMPLOYED"] = pd.cut(sub_data["Years_EMPLOYED"], 5)
print(sub_data["equal_width_Years_EMPLOYED"].value_counts().sort_index())

sub_data["equal_width_Years_EMPLOYED"] = pd.qcut(sub_data["Years_EMPLOYED"], 5)
print(sub_data["equal_width_Years_EMPLOYED"].value_counts().sort_index())

(-0.0491, 9.815]    199580
(9.815, 19.63]       39314
(19.63, 29.444]       9950
(29.444, 39.259]      2982
(39.259, 49.074]       311
Name: equal_width_Years_EMPLOYED, dtype: int64
(-0.001, 1.69]      50511
(1.69, 3.425]       50409
(3.425, 5.918]      50390
(5.918, 10.071]     50399
(10.071, 49.074]    50428
Name: equal_width_Years_EMPLOYED, dtype: int64


In [5]:
sub_data["equal_width_AMT_INCOME_TOTAL"] = pd.cut(sub_data["AMT_INCOME_TOTAL"], 5)
print(sub_data["equal_width_AMT_INCOME_TOTAL"].value_counts().sort_index())

sub_data["equal_width_AMT_INCOME_TOTAL"] = pd.qcut(sub_data["AMT_INCOME_TOTAL"], 5)
print(sub_data["equal_width_AMT_INCOME_TOTAL"].value_counts().sort_index())

(-91324.35, 23420520.0]      307510
(23420520.0, 46815390.0]          0
(46815390.0, 70210260.0]          0
(70210260.0, 93605130.0]          0
(93605130.0, 117000000.0]         1
Name: equal_width_AMT_INCOME_TOTAL, dtype: int64
(25649.999, 99000.0]       63671
(99000.0, 135000.0]        85756
(135000.0, 162000.0]       35453
(162000.0, 225000.0]       75513
(225000.0, 117000000.0]    47118
Name: equal_width_AMT_INCOME_TOTAL, dtype: int64


因為有outlier，等寬的結果很差

In [6]:
sub_data["equal_width_Years_BIRTH"] = pd.cut(sub_data["Years_BIRTH"], 5)
print(sub_data["equal_width_Years_BIRTH"].value_counts().sort_index())

sub_data["equal_width_Years_BIRTH"] = pd.qcut(sub_data["Years_BIRTH"], 5)
print(sub_data["equal_width_Years_BIRTH"].value_counts().sort_index())

(20.469, 30.238]    46806
(30.238, 39.959]    80188
(39.959, 49.679]    74661
(49.679, 59.4]      66194
(59.4, 69.121]      39662
Name: equal_width_Years_BIRTH, dtype: int64
(20.517, 32.038]    61512
(32.038, 39.496]    61504
(39.496, 47.178]    61509
(47.178, 56.093]    61491
(56.093, 69.121]    61495
Name: equal_width_Years_BIRTH, dtype: int64


#### 根據上列等寬、等頻的劃分結果，進行自定義的 bin

In [7]:
sub_data["Years_EMPLOYED_grp"] = pd.cut(sub_data["Years_EMPLOYED"], bins = [-0.0001, 2, 4, 6, 10, 50])
print(sub_data["Years_EMPLOYED_grp"].value_counts().sort_index())

(-0.0001, 2.0]    59827
(2.0, 4.0]        54728
(4.0, 6.0]        38016
(6.0, 10.0]       48612
(10.0, 50.0]      50954
Name: Years_EMPLOYED_grp, dtype: int64


In [8]:
sub_data["AMT_INCOME_TOTAL_grp"] = pd.cut(sub_data["AMT_INCOME_TOTAL"], bins = [-0.0001, 100000, 150000, 200000, 225000, 120000000])
print(sub_data["AMT_INCOME_TOTAL_grp"].value_counts().sort_index())

(-0.0001, 100000.0]        63698
(100000.0, 150000.0]       91591
(150000.0, 200000.0]       64307
(200000.0, 225000.0]       40797
(225000.0, 120000000.0]    47118
Name: AMT_INCOME_TOTAL_grp, dtype: int64


In [9]:
sub_data["Years_BIRTH_grp"] = pd.cut(sub_data["Years_BIRTH"], bins = [20, 30, 40, 50, 60, 100])
print(sub_data["Years_BIRTH_grp"].value_counts().sort_index())

(20, 30]     45021
(30, 40]     82308
(40, 50]     76541
(50, 60]     68062
(60, 100]    35579
Name: Years_BIRTH_grp, dtype: int64
